# Get best configurations for all algorithms and color spaces

In [17]:
import os
import json
import helpers as h
import evaluation
import skimage
import gradients
from skimage import color
from algorithms.base import f_score
import cv2 as cv
import numpy as np
import merging
from latex_scripts import print_values, print_best_overall

colorspaces = ['BGR','Lab','ab','YCrCb','CrCB','HSV','a*+b*','Cr+Cb','saturation']
algorithms = ['felzenszwalb', 'slic', 'watershed', 'fast_scanning', 'region_growth_simple','region_growth_var', 'region_growth_exponent']

ROOT_DIR = 'results'

def clean_config(config):
    new_config = config.copy()
    
    new_config.pop('algorithm', None)
    new_config.pop('color', None)
    new_config.pop('precision', None)
    new_config.pop('recall', None)
    new_config.pop('f_score', None)
    new_config.pop('npri_score', None)
    
    return new_config

def get_best_configurations(scores):
    best_rand = sorted(scores_algorithm, key=lambda x: x['npri_score'], reverse=True)[0]
    best_f_score = sorted(scores_algorithm, key=lambda x: x['f_score'], reverse=True)[0]
    best_precision = sorted(scores_algorithm, key=lambda x: x['precision'], reverse=True)[0]
    best_recall = sorted(scores_algorithm, key=lambda x: x['recall'], reverse=True)[0]
    best_custom = sorted(scores_algorithm, key=sort_func, reverse=True)[0]
    
    return [best_rand, best_f_score, best_precision, best_recall, best_custom]

def load_dataset():
    dataset = {}

    for algorithm in algorithms:
        algorithm_dir = os.path.join(ROOT_DIR, algorithm)
        dataset[algorithm] = {}

        for cspace in colorspaces:
            json_path = os.path.join(algorithm_dir, f"{cspace}.json")
            if os.path.isfile(json_path):
                dataset[algorithm][cspace] = json.load(open(json_path))
            else:
                print(f"[ERROR]: Json for '{algorithm}: {cspace}' not found")
    return dataset

dataset = load_dataset()
image_path = "understanding.jpg"
sort_func = lambda x: f_score(x['precision'], x['recall'], alpha=0.05)

image = skimage.io.imread(image_path, plugin='pil')
#Get resize
width = 512
(H, W) = image.shape[:2]
r = width / float(H)
dim = (int(W * r), width)
image = cv.resize(image, dim, interpolation=cv.INTER_AREA)

scores_overall = []
latex_array = []
measures_string = ""

for a in algorithms:
    scores_algorithm = []
    for c in colorspaces:
        data = [{"color": c, "algorithm": a, **config} for config in dataset[a][c]]
        scores_algorithm += data
        scores_overall += data
    configs = get_best_configurations(scores_algorithm)
    print(a)
    for c in configs:
        print(c['color'])
        print()
    print(print_values(a, configs, sort_func))
    measures_string += print_values(a, configs, sort_func)
    latex_array.extend([clean_config(config).values() for config in configs])
    
print_best_overall(scores_overall, sort_func)

felzenszwalb
ab

ab

ab

YCrCb

Cr+Cb

Felzenszwalb & 82.5\% & 52.2\% & 40.8\% & 100.0\% & 84.9\% \\ 
slic
HSV

ab

saturation

HSV

CrCB

Slic & 28.0\% & 43.9\% & 30.7\% & 99.8\% & 86.4\% \\ 
watershed
Lab

BGR

BGR

a*+b*

a*+b*

Watershed & 26.2\% & 28.6\% & 19.1\% & 94.8\% & 73.5\% \\ 
fast_scanning
CrCB

Cr+Cb

ab

YCrCb

CrCB

Fast_Scanning & 92.6\% & 72.6\% & 72.8\% & 97.8\% & 91.1\% \\ 
region_growth_simple
a*+b*

a*+b*

a*+b*

Lab

ab

Region_Growth_Simple & 90.6\% & 66.6\% & 57.1\% & 97.3\% & 90.8\% \\ 
region_growth_var
a*+b*

BGR

Cr+Cb

BGR

BGR

Region_Growth_Var & 82.1\% & 60.2\% & 100.0\% & 94.2\% & 88.5\% \\ 
region_growth_exponent
CrCB

CrCB

a*+b*

BGR

CrCB

Region_Growth_Exponent & 93.1\% & 69.3\% & 93.9\% & 98.0\% & 91.5\% \\ 
Overall results:
Best rand-index:
{'color': 'CrCB', 'algorithm': 'region_growth_exponent', 'max_diff': 6.0, 'min_size_factor': 0.0005, 'exponent': 0.8, 'npri_score': 0.9309842697902132, 'precision': 0.6065394158428989, 'recall': 0.9163492860

# Custom Merge

In [ ]:
def custom_merge(config):
    config = results[0]
    alg_class = evaluation.algorithm2class[config['algorithm']]
    converted_image = evaluation.colorspace2function[config['color']](image)
    alg = alg_class(converted_image)
    clean_config(config)
    labels = alg.run(**config)
    CrCb = h.create_2D_image(image)

    labels2 = merging.custom_merge(CrCb, labels, max_diff=4.0)

    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    out1 = color.label2rgb(labels, image=image, kind='avg')
    out2 = color.label2rgb(labels2, image=image, kind='avg')
    #out = skimage.segmentation.mark_boundaries(out, labels, color=(0,1,1))
    gradient = gradients.adaptive_threshold(h.create_sum_image(image), block_size=5, C=1)
    mask = gradient.astype(np.bool_)
    out2[mask] = 0

    h.draw_boxes(image, labels2)
    h.show_images([image, h.convert(gradient), out1, out2])

In [4]:
### Evaluate Single results
import evaluation
from algorithms import MaskRCNN

def avg_single_results(results):
    npri_sum = 0.0
    precision_sum = 0.0
    recall_sum = 0.0
    f_score_sum = 0.0
    
    num_images = len(results)
    
    for configurations in results:
        npri_sum += configurations["npri_score"]
        precision_sum += configurations["precision"]
        recall_sum += configurations["recall"]
        f_score_sum += configurations["f_score"]

    results[0]["npri_score"] = (npri_sum / num_images)
    results[0]["precision"] = (precision_sum / num_images)
    results[0]["recall"] = (recall_sum / num_images)
    results[0]["f_score"] = (f_score_sum / num_images)
    
    return results[0]

data = evaluation.get_data(lambda x: x, 512)
results = []
alg = MaskRCNN()

for img, ground_truth in data:
    results.append(alg.evaluate(img, ground_truth))
    
print(avg_result(results))


Prepared data.

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.8
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  144.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[237 382 280 427]
 [187 145 239 198]
 [234 435 278 483]
 [186 383 233 431]
 [236  88 282 140]
 [148 554 195 604]
 [427 363 465 406]
 [129 446 177 491]
 [332 382 372 426]
 [326 266 370 316]
 [ 51 501 102 551]
 [157 496 204 541]
 [183 439 228 487]
 [296 205 343 254]
 [ 21 207 109 261]
 [244 144 291 190]
 [136 388 187 436]
 [ 99 501 154 550]
 [341 209 379 255]
 [382 374 422 415]
 [384 266 423 308]
 [333 144 385 202]
 [260 482 300 535]
 [382 209 421 252]
 [207 502 253 568]
 [128  96 185 149]
 [433 255 475 300]
 [207 577 288 640]
 [427 151 468 199]
 [337  94 383 142]]
[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [

Processing 1 images
image                    shape: (512, 682, 3)         min:    1.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  135.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[170 110 233 168]
 [279 440 335 500]
 [215 446 272 500]
 [369 311 418 367]
 [370 366 421 418]
 [216 182 271 233]
 [304 179 362 224]
 [152 182 211 233]
 [313 243 371 302]
 [373 252 425 305]
 [306 311 359 362]
 [100 243 160 303]
 [184 309 248 368]
 [ 46 105 116 178]
 [139 455 198 516]
 [308 500 364 560]
 [370 118 423 174]
 [249 501 300 553]
 [251 106 306 168]
 [363 183 416 236]
 [210 370 262 426]
 [308 111 365 170]
 [372  32 423  90]
 [103 318 155 380]
 [332 439 391 497]]
[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False Fal

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  150.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[264  94 297 131]
 [205 283 232 308]
 [300 187 330 217]
 [124 103 161 140]
 [112 138 143 172]
 [146 249 176 279]
 [177 284 206 311]
 [114  55 147  92]
 [259 214 291 241]
 [309 313 336 337]
 [241 319 268 342]
 [143 280 171 305]
 [305 147 340 181]
 [321 443 345 462]
 [327 217 359 247]
 [289 223 318 250]
 [104 244 131 273]
 [118 180 150 211]
 [189 389 212 409]
 [256 245 285 271]
 [180 251 208 280]
 [276 297 302 320]
 [188 367 212 388]
 [311 285 338 311]
 [137 445 160 463]
 [145 319 171 342]
 [299 336 329 362]
 [341 269 373 296]
 [378 267 408 294]
 [158 369 183 390]
 [103 394 127 413]
 [190 408 213 428]
 [275 325 302 348]
 [278 390 303 408]
 [220 414 243 432]
 [182 209 209 252]
 [335 179 369 213]
 [108 271

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[184 396 214 429]
 [134 439 164 471]
 [267 277 294 305]
 [ 86 268 119 318]
 [ 82 341 118 397]
 [255 187 284 216]
 [353 194 381 223]
 [154 287 184 317]
 [152 167 181 199]
 [188 269 219 299]
 [371 490 402 517]
 [157 257 187 286]
 [121 171 153 204]
 [163 441 192 472]
 [407 526 433 554]
 [429 489 452 515]
 [153 197 183 229]
 [121 141 152 173]
 [363  97 393 128]
 [341 492 372 521]
 [162 615 190 644]
 [327  56 355  89]
 [385 320 411 346]
 [354 366 381 394]
 [ 81 193 116 245]
 [123 272 154 303]
 [363 430 392 459]
 [277 372 306 401]
 [364 402 394 431]
 [255 465 282 496]
 [120  98 155 132]
 [168 346 199 377]
 [259 156 288 186]
 [287 602 317 629]
 [300 431 326 460]
 [136 571 165 602]
 [228 152 258 184]
 [372 557

...

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  144.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[]
[]
[]
[]
Processing 1 images
image                    shape: (512, 682, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  150.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[432 180 510 256]
 [187  71 247 141]
 [446 103 510 177]
 [298 401 357 467]
 [239 426 292 488]
 [362 438 429 500]
 [297 457 353 519]
 [179 224 236 283]
 [189 436 234 489]
 [355 493 414 551]
 [199 489 246 543]
 [361 551 419 614]
 [415 516 481 586]
 [184 148 243 214]
 [296 558 349 616]
 [253 121 321

[[374 265 421 312]
 [149 226 226 279]
 [ 69 312 111 359]
 [346 464 396 513]
 [190   5 248  64]
 [228 260 273 307]
 [262 571 301 609]
 [395 471 440 515]
 [380 105 432 152]
 [314 373 363 418]
 [363 375 410 426]
 [211 581 251 621]
 [230 208 274 257]
 [259 355 305 400]
 [330  46 381  99]
 [ 46 363  87 410]
 [210 543 249 578]
 [277 255 324 309]
 [329 211 377 257]
 [328 263 371 301]
 [ 70 411 112 450]
 [375 213 423 259]
 [277 210 323 253]
 [321 110 381 156]
 [ 93 369 133 411]
 [379  41 430 100]
 [109 310 182 375]]
[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False F

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[274 349 312 389]
 [225 302 262 341]
 [199 164 239 206]
 [143 255 181 296]
 [231 464 270 525]
 [229 343 267 382]
 [310 310 347 349]
 [128 436 164 479]
 [429 245 468 284]
 [431 506 468 546]
 [424 304 458 340]
 [ 21  60  67 131]
 [381 467 417 522]
 [235 423 272 462]
 [103 168 146 211]
 [ 38 374  80 414]
 [ 30 139  72 182]
 [143 298 184 339]
 [223 249 261 289]
 [ 35 328  80 369]
 [ 24  12  67  60]
 [ 28 190  70 234]
 [ 87 296 127 338]
 [ 34 288  75 331]
 [ 25 509  71 577]
 [422 410 458 467]
 [ 80   5 118  53]
 [ 84 539 124 605]
 [390 536 426 596]
 [ 85 489 123 533]
 [404 138 442 179]]
[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False Fal

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  133.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[313 428 342 457]
 [180  65 211  97]
 [291 268 319 296]
 [384 242 413 272]
 [147  56 174  88]
 [313 348 342 377]
 [266 545 298 577]
 [253 473 282 503]
 [ 56  38  86  69]
 [124 439 152 469]
 [143 214 174 246]
 [256 449 284 474]
 [283 401 310 431]
 [260  61 289  93]
 [283 372 310 401]
 [208 550 235 578]
 [174 387 203 415]
 [318 269 346 298]
 [109 531 138 563]
 [345 196 373 227]
 [147 156 177 185]
 [239 537 269 569]
 [119  91 150 123]
 [293  94 321 127]
 [168 565 199 594]
 [336 538 366 570]
 [344 166 375 200]
 [276  31 304  61]
 [149  90 178 122]
 [149 127 177 156]
 [292 158 320 189]
 [ 93 460 124 488]
 [172 530 199 564]
 [382 276 410 303]
 [104 325 134 358]
 [ 87  55 117  87]
 [290 127 320 159]
 [141 537

molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  108.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[[164 384 304 478]
 [158 292 321 377]
 [302 482 456 582]
 [171 192 314 279]
 [318 291 464 378]
 [329 387 465 476]
 [324 197 467 284]]
[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False Fal

In [6]:
from algorithms.base import f_score

f_score(0.859066690677199, 0.7781856586811545, alpha=0.05)

0.7818662898295046

In [1]:
from algorithms import MaskRCNN
import skimage
import cv2 as cv
from skimage import color
import helpers as h

image_path = "understanding.jpg"
image = skimage.io.imread(image_path, plugin='pil')
#Get resize
width = 512
(H, W) = image.shape[:2]
r = width / float(H)
dim = (int(W * r), width)
image = cv.resize(image, dim, interpolation=cv.INTER_AREA)

alg = MaskRCNN()
labels = alg.detect(image)

out = color.label2rgb(labels, image=image, kind='avg')
h.show_images([image, out])


Using TensorFlow backend.



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.8
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [12]:
txt_file = "hello/world/path/name.json"
directorr = txt_file.split('/')[-2]
txt_file = txt_file.replace(directorr, 'labels')
print(txt_file)

hello/world/labels/name.json
